# Experimenting to fix cracks & folds in basil  
Using the Optimizer to fix cracks & folds in basil

In [3]:
!mkdir -p ~/.cloudvolume/secrets
!cp /notebooks/*.json ~/.cloudvolume/secrets/
!pip install cloud-volume
!pip install scikit-image

    100% |################################| 71kB 2.2MB/s ta 0:00:011
    100% |################################| 133kB 3.2MB/s ta 0:00:01
    100% |################################| 194kB 3.2MB/s ta 0:00:01
    100% |################################| 81kB 4.7MB/s ta 0:00:011
    100% |################################| 92kB 4.3MB/s ta 0:00:011
    100% |################################| 51kB 7.2MB/s eta 0:00:01
    100% |################################| 419kB 2.5MB/s ta 0:00:01
    100% |################################| 92kB 5.3MB/s ta 0:00:011
    100% |################################| 143kB 4.7MB/s ta 0:00:01
    100% |################################| 6.4MB 235kB/s eta 0:00:01
    100% |################################| 133kB 9.8MB/s eta 0:00:01
    100% |################################| 71kB 12.8MB/s ta 0:00:01
    100% |################################| 61kB 8.4MB/s eta 0:00:01
    100% |################################| 4.2MB 368kB/s eta 0:00:01
    100% |#####################

    100% |################################| 614kB 2.4MB/s eta 0:00:01
    100% |################################| 51kB 10.9MB/s ta 0:00:01
    100% |################################| 61kB 11.2MB/s ta 0:00:01
    100% |################################| 266kB 5.6MB/s eta 0:00:01
    100% |################################| 71kB 12.7MB/s ta 0:00:01
    100% |################################| 143kB 10.0MB/s ta 0:00:01
    100% |################################| 92kB 12.3MB/s ta 0:00:01
    100% |################################| 71kB 11.7MB/s ta 0:00:01
    100% |################################| 51kB 13.5MB/s ta 0:00:01
    100% |################################| 81kB 12.2MB/s ta 0:00:01
    100% |################################| 61kB 12.0MB/s ta 0:00:01
    100% |################################| 61kB 12.4MB/s ta 0:00:01
    100% |################################| 61kB 12.0MB/s ta 0:00:01
    100% |################################| 153kB 8.1MB/s eta 0:00:01
    100% |####################

    100% |################################| 51kB 12.1MB/s ta 0:00:01
    100% |################################| 51kB 10.1MB/s ta 0:00:01
    100% |################################| 552kB 2.9MB/s eta 0:00:01
    100% |################################| 102kB 12.6MB/s a 0:00:01
    100% |################################| 204kB 6.8MB/s eta 0:00:01
    100% |################################| 81kB 12.1MB/s ta 0:00:01
    100% |################################| 51kB 12.2MB/s ta 0:00:01
    100% |################################| 829kB 1.9MB/s eta 0:00:01
    100% |################################| 163kB 8.4MB/s eta 0:00:01
    100% |################################| 71kB 12.4MB/s ta 0:00:01
    100% |################################| 9.0MB 173kB/s eta 0:00:01
    100% |################################| 81kB 11.8MB/s ta 0:00:01


    100% |################################| 225kB 6.4MB/s eta 0:00:01
  Running setup.py bdist_wheel for posix-ipc ... done
  Stored in directory: /root/.cache/pip/wheels/5a/5c/17/d0e2d421abaf3b4097bf9db11108380d734195c6d15c24269d
  Running setup.py bdist_wheel for psutil ... done
  Stored in directory: /root/.cache/pip/wheels/fe/33/84/baea3d665de2d1af7e8f827f3883811bba5e4149443ccf8191
  Running setup.py bdist_wheel for tenacity ... done
  Stored in directory: /root/.cache/pip/wheels/cf/dd/89/99693d2e2c5954ccd7b7ba4fe9571b385712939d25e584a03a
  Running setup.py bdist_wheel for nose2 ... done
  Stored in directory: /root/.cache/pip/wheels/66/65/52/e7e6215ebee79eff0f3a60db0e6e6f6d751c3ebbb7a52cdfbd
  Running setup.py bdist_wheel for blosc ... done
  Stored in directory: /root/.cache/pip/wheels/fa/2a/00/8ff3a1d3d423a3163b93917a72146e85c3a3484ba6574854a5
  Running setup.py bdist_wheel for pandocfilters ... done
  Stored in directory: /root/.cache/pip/wheels/fc/a0/4c/ec26eaa3f2253144e29941b

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 35.5MB 39kB/s  eta 0:00:01
    100% |################################| 1.6MB 986kB/s eta 0:00:01
    100% |################################| 5.7MB 263kB/s eta 0:00:01
  Running setup.py bdist_wheel for networkx ... done
  Stored in directory: /root/.cache/pip/wheels/44/c0/34/6f98693a554301bdb405f8d65d95bbcd3e50180cbfdd98a94e
Successfully built networkx
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
from cloudvolume import CloudVolume
from cloudvolume.lib import Bbox
import numpy as np
import torch
from torch.autograd import Variable
from optimize import Optimizer
from copy import deepcopy
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
img_mip = 2
mask_mip = 5
src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                parallel=2, fill_missing=True)
cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                 parallel=2, fill_missing=True)
folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                 parallel=2, fill_missing=True)

In [5]:
def use_src_mask_and_one_dst():
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_offset = np.array([102716, 107077, 526])
    full_size = np.array([2048, 2048, 1])
    full_bbox = Bbox(full_offset, full_offset+full_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_bbox))
    dst_bbox = src_bbox + [0,0,1]
    mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_bbox))
    print(src_bbox)
    print(dst_bbox)
    print(mask_bbox)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_with_mask", mip=img_mip, 
                        info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    dst[dst_bbox.to_slices()] = src[dst_bbox.to_slices()]
    src_img = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    dst_img = src[dst_bbox.to_slices()][:,:,0,0] / 255.0

    fold = folds[mask_bbox.to_slices()][:,:,0,0] / 255.0
    crack = cracks[mask_bbox.to_slices()][:,:,0,0] / 255.0
    mask = resize(np.minimum(fold, crack), src_bbox.size3()[:2])
    field = flow.process(src_img, [dst_img], mask, [np.ones_like(dst_img)])
    pred = flow.render(src_img, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

Bbox([25679, 26769, 526],[26191, 27281, 527])
Bbox([25679, 26769, 527],[26191, 27281, 528])
Bbox([3209, 3346, 526],[3273, 3410, 527])


In [11]:
def use_masks_and_two_dsts():
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [15]:
def use_masks_and_four_dsts():
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-2,-1,1,2]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_four_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [32]:
def use_filtered_mask_and_two_dsts():
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_filtered_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    src_mask *= (src_mask < 1/255.0) | (src_mask > 10/255.0)
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [34]:
def use_filtered_mask_and_two_dsts2():
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_filtered_mask_two_dsts2", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    src_mask *= (src_mask < 4/255.0) | (src_mask > 10/255.0)
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [11]:
def use_masks_and_two_dsts_mip1():
    img_mip = 1
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_src_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts_mip1", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field
    

In [13]:
def use_masks_and_two_dsts_mip0():
    img_mip = 0
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_src_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts_mip0", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field
    

In [24]:
def use_masks_and_two_dsts_crack():
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102320, 111106, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_src_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    src_mask = np.ones_like(src_mask) * ((src_mask < 6/255.0) | (src_mask > 20/255.0))
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field

In [26]:
use_masks_and_two_dsts_crack()

Downloading:   0%|          | 0/2 [00:00<?, ?it/s]

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  4.27it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  4.63it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  3.11it/s]

Downloading:   0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
Downloading:   0%|          | 0/2 [00:00<?, ?it/s]

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]

(512, 512) 2 (512, 512) 2
Downsample factor 16x
[0.1669654]
[0.15276602]
[0.14370957]
[0.13721247]
[0.13196947]
[0.1276289]
[0.12399141]
[0.12095544]
[0.11832531]
[0.11608969]
[0.11408599]
[0.11234529]
[0.11078461]
[0.10942483]
[0.1081812]
[0.10712405]
[0.10611636]
[0.105271]
[0.10445688]
[0.10377265]
[0.1030516]
[0.10248823]
[0.10187016]
[0.1013606]
[0.10086738]
[0.10037912]
[0.0999532]
[0.09955811]
[0.09915709]
[0.09880945]
[0.09848557]
[0.0981506]
[0.09787173]
[0.09757188]
[0.09730653]
[0.09706388]
[0.09681455]
[0.09658114]
[0.09638454]
[0.09614894]
[0.09597133]
[0.09575448]
[0.09559987]
[0.09537506]
[0.09525824]
[0.09505308]
[0.09492615]
[0.09475937]
[0.09460657]
[0.09447077]
[0.09433492]
[0.09419902]
[0.09407201]
[0.09394701]
[0.0938204]
[0.09370501]
[0.09358607]
[0.0934899]
[0.09335557]
[0.09328181]
[0.09313314]
[0.09308398]
[0.09295058]
[0.09287048]
[0.09277383]
[0.09267186]
[0.09259929]
[0.09250891]
[0.09242742]
[0.09234631]
[0.09225085]
[0.092195]
[0.09209837]
[0.09203881]
[0.

[0.08567376]
[0.08567169]
[0.0856826]
[0.08564918]
[0.08569657]
[0.08564822]
[0.08568618]
[0.08565556]
[0.08567392]
[0.08566672]
[0.08565041]
[0.08567847]
[0.08565192]
[0.08565814]
[0.08566509]
[0.08566007]
[0.08565702]
[0.08564855]
[0.0856647]
[0.08563852]
[0.08566508]
[0.08565393]
[0.08564951]
[0.08565202]
[0.08563624]
[0.08565769]
[0.08564408]
[0.08564997]
[0.08564617]
[0.08563381]
[0.08564955]
[0.08563682]
[0.08564956]
[0.08563234]
[0.08563781]
[0.08564308]
[0.0856443]
[0.0856336]
[0.08562648]
[0.08564029]
[0.08562887]
[0.08563368]
[0.08564306]
[0.08561465]
[0.08563402]
[0.08561414]
[0.08565313]
[0.08560663]
[0.08563197]
[0.08562568]
[0.08561122]
[0.0856397]
[0.0856095]
[0.0856277]
[0.08561984]
[0.0856116]
[0.08562528]
[0.08561001]
[0.08562762]
[0.08560326]
[0.08562163]
[0.08561727]
[0.085601]
[0.08562304]
[0.08559982]
[0.0856258]
[0.08557612]
[0.08563101]
[0.08557591]
[0.08561782]
[0.08558628]
[0.08560026]
[0.08559018]
[0.08559471]
[0.08559056]
[0.08558568]
[0.08557834]
[0.0856038

[0.20811543]
[0.20812096]
[0.20813048]
[0.20810665]
[0.20811024]
[0.20811889]
[0.20808087]
[0.20812523]
[0.20809217]
[0.20811406]
[0.20808503]
[0.20810115]
[0.20807233]
[0.20808509]
[0.20810702]
[0.20806354]
[0.20807365]
[0.20808744]
[0.20806184]
[0.20806679]
[0.208069]
[0.20807254]
[0.20805676]
[0.20806977]
[0.2080462]
[0.20805243]
[0.20803903]
[0.20806098]
[0.20803197]
[0.20806748]
[0.20802012]
[0.20804287]
[0.20802873]
[0.20803419]
[0.20803091]
[0.20801884]
[0.20803273]
[0.2080084]
[0.20802465]
[0.20800391]
[0.20800513]
[0.2080333]
[0.20797677]
[0.20803088]
[0.20800242]
[0.20800361]
[0.20798686]
[0.20800243]
[0.20799091]
[0.20798263]
[0.20799804]
[0.20797116]
[0.20798023]
[0.20799896]
[0.20795959]
[0.2079877]
[0.20796788]
[0.20797905]
[0.20794444]
[0.20797366]
[0.20795146]
[0.20793447]
[0.20796987]
[0.20795213]
[0.2079301]
[0.20795715]
[0.20792243]
[0.20795724]
[0.207928]
[0.20793925]
[0.20792268]
[0.20792018]
[0.20792696]
[0.20790248]
[0.20793667]
[0.20792204]
[0.20790029]
[0.20791

[0.86127484]
[0.8611895]
[0.86116564]
[0.86113715]
[0.8610953]
[0.86102223]
[0.8610276]
[0.8609462]
[0.8609171]
[0.8608489]
[0.8608434]
[0.86076176]
[0.86075234]
[0.86066055]
[0.8606543]
[0.86062384]
[0.86053956]
[0.8605124]
[0.8604811]
[0.86045873]
[0.86037576]
[0.860377]
[0.8603389]
[0.86026496]
[0.8602272]
[0.86021763]
[0.86011076]
[0.8601779]
[0.8600514]
[0.8600777]
[0.860039]
[0.86001366]
[0.8599491]
[0.8599075]
[0.8599157]
[0.8598387]
[0.859822]
[0.8597502]
[0.85977644]
[0.8597032]
[0.85969555]
[0.85962665]
[0.85966706]
Downsample factor 1x
[1.934968]
[1.5964333]
[1.4949934]
[1.453404]
[1.4320784]
[1.4194832]
[1.411073]
[1.4050227]
[1.4003263]
[1.3961629]
[1.3928525]
[1.3900906]
[1.3876561]
[1.3856001]
[1.3836515]
[1.3819278]
[1.3803778]
[1.3789687]
[1.377528]
[1.3762425]
[1.3752158]
[1.3742425]
[1.3731608]
[1.3721654]
[1.3713628]
[1.3705587]
[1.3697078]
[1.3690088]
[1.3682923]
[1.3678169]
[1.3669654]
[1.3664323]
[1.365968]
[1.3654311]
[1.3649416]
[1.364534]
[1.3639715]
[1.363609

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Downloading:   0%|          | 0/4 [00:00<?, ?it/s]

[1.3539577]
[1.3539131]
[1.353694]
[1.3536055]
[1.3535408]
[1.3533934]
[1.3532147]
[1.3532126]
[1.3530694]
[1.3530371]
[1.3528206]
[1.3527346]
[1.3525803]
[1.352448]
[1.352418]
[1.3523213]
[1.35213]
[1.3522346]
[1.3519905]
[1.3520331]
done: 2146
torch.Size([1, 512, 512, 2])



Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  9.08it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  8.79it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  9.87it/s]



array([[[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       ...,

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]]], dtype=float32)